In [1]:
import pandas as pd
import twint
import nest_asyncio
nest_asyncio.apply()
import os
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords

In [2]:
tags_tweet = ["#KeikoNova", "#keikonuncamas", "#FujimoriNuncaMás", "#NoAKeiko", "#Keikino", "#KeikoTAPERdiendo",
              "#GolpeElectoral","#FuerzaPopular", "#ConMiVotoNoTeMetas",
               "#BICENTENARIOMEJORCONSTITUCION", "#FujimafiaNuncaMás", "#NoAlNarcoEstado", "#FujimorismoNuncaMas",
               "#KeikoGolpistaYDictadora", "#keikonoimpugnesmiacta", "#KEIKOMENTIROSA", "#KeikoPresidenta2021",
               "#KeikoPresidenta","#PedroCastilloPresidente","#PedroCastilloNoVa", "#NoAlComunismoEnElPeru",
               "#NoAlTerrorismo", "#NoAlComunismo","#LosBuenosSomosMas","#pedrocastilloNICagando", "#PedroCastilloNoVa", 
              "#PedroTiraPiedras","#Genocidas","#KeikoPresidenta2021","#KeikoFujimori","#CastilloMachistayHomofobico",
              "#CastilloYCerronUnSoloCorazon","#EleccionesPerú2021","#DefiendeTuVoto","#RespetaMiVoto","#LaMaldiciondelaSenoraK",
              "#LaCamisetaNoSeMancha","#FujimorismoNuncaMás","#SegundaVuelta", "#EleccionesBicentenario","#Elecciones2021PE",
               "#PeruDecide2021","#PedroCastillo","#KeikoFujimori","#Elecciones2021","#eleccionesperu","#ONPE","#EleccionesPeru", 
              "#Elecciones2021Peru","#EleccionesBicentenario", "#Onpe","ONPE","#JNE","#SegundaVuelta", 
              "#ONPEPeru","#JNEPeru","#PeruLibre","#PERULIBRE","#TerrorismoNuncaMas","#Jne", "#PedroCastilloTe", "#JNE_Peru"]

In [3]:
tags_tweet1 = ["#KeikoNova","#SegundaVuelta","#PedroCastilloTe",'#NoAlNarcoEstado','#RespetaMiVoto',
       '#KeikoPresidenta','#FujimafiaNuncaMas',"#pedrocastilloNiCagando",'#KEIKOMENTIROSA',
       '#PedroCastilloNoVa','#DefiendeTuVoto','#DefiendeTuVoto','#PedroCastilloPresidente',
       '#keikonuncamas','#BICENTENARIOMEJORCONSTITUCION','#NoAKeiko','#KeikoTAPERdiendo','#CastilloYCerronUnSoloCorazon',
       '#NoalTerrorismo','#KeikoGolpistaYDictadora','#NoAlComunismoEnElPeru','#PeruDecide2021',
       '#FujimoriNuncaMas','#LosBuenosSomosMas','#KeikoFujimori','#FujimorismoNuncaMas','#Genocidas',
       '#NoAlComunismo','#PeruLibre','#PedroCastillo','#Elecciones2021','#TerrorismoNuncaMas']


In [4]:
tags = ["#KeikoNova","#SegundaVuelta","#PedroCastilloTe",'#NoAlNarcoEstado','#RespetaMiVoto',
       '#KeikoPresidenta','#FujimafiaNuncaMas',"#pedrocastilloNiCagando",'#KEIKOMENTIROSA',
       '#PedroCastilloNoVa','#DefiendeTuVoto','#DefiendeTuVoto','#PedroCastilloPresidente',
       '#keikonuncamas','#BICENTENARIOMEJORCONSTITUCION','#NoAKeiko','#KeikoTAPERdiendo','#CastilloYCerronUnSoloCorazon',
       '#NoalTerrorismo','#KeikoGolpistaYDictadora','#NoAlComunismoEnElPeru','#PeruDecide2021',
       '#FujimoriNuncaMas','#LosBuenosSomosMas','#KeikoFujimori','#FujimorismoNuncaMas','#FujimorismoNuncaMás','#Genocidas',
       '#NoAlComunismo','#PeruLibre','#PedroCastillo','#Elecciones2021','#TerrorismoNuncaMas','#FuerzaPopular','#FujimoriNuncaMás',
       'KeikoFujimori','PedroCastillo','keiko','castillo','EleccionesPresidenciales2021','JNE','ONPE','Derecha','Izquierda','DebatePresidencial','#Rojos']

tag_vac = ['#VacúnateYa','#COVID19', '#PongoElHombro', '#LaVacunaEsVida', '#NoBajemosLaGuardia', '#VamosaSalirAdelante', '#vacunacion',
           '#sialavacuna', '#VacunaFest','#TodasLasVacunasSirven', '#NuevaCepa', '#Sinopharm', '#YoApoyoaBeto', '#ErnestoBustamante',
           '#SagastiGenocida', '#VacunaCOVID19', '#SuizaLab', '#AstraZeneca', '#Pfizer', '#delta', '#variante', '#PfizerBiontech', 
           '#Moderna', '#CureVac']
tag_vac1 = ['#VacunateYa']
tag2 = ['#MirthaVásquez','#Premier','#GabineteVásquez' ,'#GuidoBellido','@patriciagamarra']

In [5]:
#prueba = ["#PeruLibre","#PERULIBRE","#TerrorismoNuncaMas","#Jne"]
#prueba=["#Elecciones2022 "#ONPE"]


In [6]:
def Crear_carpeta(Rutabase, name_carpeta):
    # Se define el nombre de la carpeta o directorio a crear
    directorio = Rutabase + name_carpeta
    if os.path.exists(directorio):
        shutil.rmtree(directorio)
        os.makedirs(directorio)
        print("Se reemplazo el directorio %s " % directorio)
    else:
        try:
            os.mkdir(directorio)
        except OSError:
            print("La creación del directorio %s falló" % directorio)
        else:
            print("Se ha creado el directorio: %s " % directorio)

In [7]:
def Generar_Tweets(lista, Rutabase, since, until):
    df_Final = pd.DataFrame()
    numero_tweets = 0
    for i in lista:
        c = twint.Config()
        #print(i)
        c.Search = i
        c.Language = 'es'
        c.Since = since
        c.Until = until
        c.Limit = 10000
        c.Pandas = True
        c.Hide_output= True
        twint.run.Search(c)
        #print(twint.storage.panda.Tweets_df)
        df_twint = twint.storage.panda.Tweets_df
        #print(df_twint)
        #print(len(df_twint))
        numero_tweets = numero_tweets + len(df_twint)
        Rutarel =  i + '.csv'
        Rutasol = os.path.join(Rutabase, Rutarel)
        df_twint.to_csv(Rutasol, index = False)
        print('Se ha creado correctamente el ' + i + '.csv, recuperando ' + str(numero_tweets) + ' tweets')
    #df_Final.to_csv("Tweets_prueba.csv", index=False)

In [72]:
def SeleccionarTweets(lista, Rutabase):
    total_df = pd.DataFrame() 
    count = 0
    for i in lista:
        Rutarel =  i + '.csv'
        Rutasol = os.path.join(Rutabase, Rutarel)
        #print(Rutasol)
        df = pd.read_csv(Rutasol, delimiter=',')
        #print(df)
        new_df = df[df['language'] == 'es']
        #print(new_df)
        #count = count + len(df)
        total_df = pd.concat([total_df, new_df],ignore_index=False)
    #print(total_df.shape)
    total_df = total_df.reset_index(drop = True)
    print('Se ha recuperado ' + str(len(total_df)) + ' tweets en el idioma espanhol' )
    return total_df
   

#df[(df['date'] > '2021-04-01') & (df['date'] < '2021-10-01')]

In [73]:
def Recuperar_Indice(lista_indices):
    index = []
    for i in lista_indices:
        index.append(i[1])
    index = pd.unique(index)
    index = list(index)
    return index

In [80]:
def Eliminar_Repetidos(df, label, language, threshold):
    tfidf = TfidfVectorizer(stop_words = stopwords.words(language))
    # Construct the TF-IDF matrix
    tfidf_matrix = tfidf.fit_transform(df[label])

    # Generate the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    print(cosine_sim)
    #Lista que recupera los tweets
    A = []
    for i in range(0,len(cosine_sim)):
        for k in range(0,len(cosine_sim)):
            if ((k>i) and (cosine_sim[i][k] >= threshold)):
                #print(k)
                A.append([i,k])
                
    indices_tweets = Recuperar_Indice(A)
    df = df.drop(indices_tweets)
    #print(df)
    df = df.reset_index(drop = True)
    print('Se han recuperado ' + str(len(df)) + ' tweets')
    return df
    

In [81]:
Rutabase = r'C:/Users/USER/Documents/Git/SATwint/'
#carpeta = 'data4/'
def Obtener_dataset(lista_tweet, name_carpeta, label, language, name):
    Crear_carpeta(Rutabase, name_carpeta)
    Ruta = Rutabase + name_carpeta
    Generar_Tweets(lista_tweet, Ruta,'2020-01-01', '2021-11-01')
    df = SeleccionarTweets(lista_tweet, Ruta)
    #print(df)
    df = Eliminar_Repetidos(df, label, language, 0.5)
    df.to_csv(name + '.csv')
    return df

In [ ]:
final_df = Obtener_dataset(tag_vac,'data5/', 'tweet', 'spanish', 'df_vacunacion' )

Se reemplazo el directorio C:/Users/USER/Documents/Git/SATwint/data5/ 
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Se ha creado correctamente el #VacúnateYa.csv, recuperando %s 2283 tweets
Se ha creado correctamente el #COVID19.csv, recuperando %s 12298 tweets
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Se ha creado correctamente el #PongoElHombro.csv, recuperando %s 13632 tweets
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Se ha creado correctamente el #LaVacunaEsVida.csv, recuperando %s 14335 tweets
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Se ha creado correctamente el #NoBajemosLaGuardia.csv, recuperando %s 21854 tweets
Se ha creado correctamente el #VamosaSalirAdelante.csv, recuperando %s 31855 tweets
